In [1]:
from cdcqr.data.glassnode import glassnode_data as gnd
from cdcqr.ct.utils import plot2
import pandas as pd
import cdcqr.analytics.utils as autil 
from importlib import reload
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
from cdcqr.backtest.vbt.runsignal import runsignal
import numpy as np
import os
from cdcqr.common.utils import LOCAL_DATA_DIR
import matplotlib.pyplot as plt
import seaborn as sns
from cdcqr.analytics.utils import calculate_corr_pvalues
from cdcqr.analytics.utils import pctrank
import time
from IPython.display import Audio
sound_file = '/core/tmp/ding.wav'
%matplotlib inline

In [2]:
gn = gnd.GlassnodeData()

### BTC

In [3]:
fs =  ['FuturesFundingRatePerpetual']
a = ['BTC']
#i = '10m'
df = gn.load_features(fs,a)
df.head(2)

loading derivatives/futures_funding_rate_perpetual asset=BTC, resolution=10m


,BTC-FuturesFundingRatePerpetual
t,
2019-12-31 16:00:00,0.000452
2019-12-31 16:10:00,0.000337


In [4]:
fs =  ['price_usd_close',]
a = ['BTC','ETH']
#i = '10m'
df = gn.load_features(fs,a)
df.head(2)

loading market/price_usd_close asset=BTC, resolution=10m
loading market/price_usd_close asset=ETH, resolution=10m


,BTC-price_usd_close,ETH-price_usd_close
t,,
2010-07-17 23:00:00,0.04951,NaN
2010-07-17 23:10:00,NaN,NaN


In [6]:
df

,BTC-price_usd_close,ETH-price_usd_close
t,,
2010-07-17 23:00:00,0.049510,NaN
2010-07-17 23:10:00,NaN,NaN
2010-07-17 23:20:00,NaN,NaN
2010-07-17 23:30:00,NaN,NaN
2010-07-17 23:40:00,NaN,NaN
...,...,...
2022-03-01 14:20:00,44515.759616,3017.446170
2022-03-01 14:30:00,44188.644401,2991.686076
2022-03-01 14:40:00,44166.210123,2994.475946


In [ ]:
cols = list(df.columns)
cols

In [ ]:
df1 = df[df.index>='20200101']
display(df1.head(2))
df1.plot2(cols[0],cols[1:3])

In [ ]:
df3 = df1.copy()
df3['ret'] = df3['BTC-price_usd_close'].pct_change()
df3['r+5'] = df3['BTC-price_usd_close'].pct_change(periods=5).shift(-5)
df3['r+10'] = df3['BTC-price_usd_close'].pct_change(periods=10).shift(10)
df3['r+100'] = df3['BTC-price_usd_close'].pct_change(periods=100).shift(100)

In [ ]:
df4 = df3.dropna()[['BTC-FuturesFundingRatePerpetual', 'ETH-FuturesFundingRatePerpetual',  'ret', 'r+5', 'r+10', 'r+100']]

In [ ]:
df4a = df4.where(~np.isinf(df4), 0)
fig, ax = plt.subplots()
sns.heatmap(df4a.corr(method='pearson'), annot=True, fmt='.4f', 
            cmap=plt.get_cmap('coolwarm'), cbar=False, ax=ax)

In [ ]:
calculate_corr_pvalues(df4a)

In [ ]:
lookback=4000
t0 = time.time()
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
df4a['BTC-FuturesFundingRatePerpetual_pctrank4k'] = df4a['BTC-FuturesFundingRatePerpetual'].rolling(lookback).apply(pctrank)
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
df4b = df4a[df4a.index>='20210101']
feature = 'BTC-FuturesFundingRatePerpetual_pctrank4k'
df4b_gp = df4b.groupby(pd.cut(df4b[feature], 50)).agg({'ret': lambda x: x.mean(skipna=True),
                                                                 'r+5': lambda x: x.mean(skipna=True), 
                                                                 'r+10': lambda x: x.mean(skipna=True),
                                                                 'r+100': lambda x: x.mean(skipna=True)})
df4b_gp.plot()

## backtest

In [ ]:
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))

In [ ]:
freq='1Min'
p=price.resample('1Min').last().ffill()
request = ['pf','backtest']

### Long BTC-FuturesFundingRatePerpetual_pctrank4k

In [ ]:
feature = 'BTC-FuturesFundingRatePerpetual_pctrank4k'
side = 'long'

t0 = time.time()
signalid = '{}-{}'.format(side, feature)
upper_qtls = [0.995, 0.99, 0.975, 0.96]
if side == 'long':
    short=False
    qtls = upper_qtls
elif side == 'short':
    short=True
    qtls = [1-x for x in upper_qtls]
else:
    raise()

res=[]
price = p.copy()
counter0 = 0
for qtl in qtls:
    if not short:
        signal = df4b[feature]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
        signal = signal.reindex(p.index).ffill()
    else:
        signal = df4b[feature]<qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
    signal = signal.reindex(p.index).ffill()
    for tp in [3,5,7, 10, 15, 20]:
        for sl in [0.5, 1, 2, 5, 10, 15, 20]:
            counter0 = counter0 + 1
            print(f'{counter0} : tp:{tp} sl:{sl} id:{signalid}')
            ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=None,n2=None,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])
            for k in ressig:
                resd={"signalid":signalid,"qtl":qtl,"short":short,"tp":tp,"sl":sl,'k':k,'sr':ressig[k].sharpe_ratio(),'n1':n1,'n2':n2,'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/46.5} # rescale to number of weeks
                res.append(resd)
                        
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
res_df = pd.DataFrame(res)
res_df.save('{}-{}_res_df'.format(side, feature))

In [ ]:
res_df.pv()

In [ ]:
tp=5
sl=20
n1=None
n2=None
qtl=0.99
short=False
pf = 'rsi_tpsl_pf'
signal = df4b[feature]>qtl 
signal = signal.reindex(p.index).ffill()

ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,
                 rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])

In [ ]:
ressig[pf].plot(subplots=['drawdowns','trade_pnl','cum_returns'])

### Short BTC-FuturesFundingRatePerpetual_pctrank4k

In [ ]:
feature = 'BTC-FuturesFundingRatePerpetual_pctrank4k'
side = 'short'

t0 = time.time()
signalid = '{}-{}'.format(side, feature)
upper_qtls = [0.995, 0.99, 0.975, 0.96]
if side == 'long':
    short=False
    qtls = upper_qtls
elif side == 'short':
    short=True
    qtls = [round(1-x,4) for x in upper_qtls]
else:
    raise()

res=[]
price = p.copy()
counter0 = 0
for qtl in qtls:
    if not short:
        signal = df4b[feature]<qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
        signal = signal.reindex(p.index).ffill()
    else:
        signal = df4b[feature]<qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
    signal = signal.reindex(p.index).ffill()
    for tp in [3,5,7, 10, 15, 20]:
        for sl in [0.5, 1, 2, 5, 10, 15, 20]:
            counter0 = counter0 + 1
            print(f'{counter0} : tp:{tp} sl:{sl} id:{signalid}')
            ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=None,n2=None,rsil=30,rsih=50,rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])
            for k in ressig:
                resd={"signalid":signalid,"qtl":qtl,"short":short,"tp":tp,"sl":sl,'k':k,'sr':ressig[k].sharpe_ratio(),'n1':n1,'n2':n2,'tr':ressig[k].total_return(),'ntrades':ressig[k].trades.count()/46.5} # rescale to number of weeks
                res.append(resd)
                        
print(time.time()-t0)
Audio(sound_file, autoplay=True)

In [ ]:
res_df = pd.DataFrame(res)
res_df.save('{}-{}_res_df'.format(side, feature))

In [ ]:
res_df.pv()

In [ ]:
tp=3
sl=5

qtl=0.01
short=True
pf = 'rsi_tpsl_pf'
signal = df4b[feature]<qtl 
signal = signal.reindex(p.index).ffill()

ressig=runsignal(price,signal=signal,tp=tp,sl=sl,ts=sl,n1=n1,n2=n2,rsil=30,rsih=50,
                 rsilag=14,maf=5,mas=15,short=short,size=np.inf,fees=0.0007,freq='1Min',init_cash=10000,request=['pf','backtest'])

In [ ]:
ressig[pf].plot(subplots=['drawdowns','trade_pnl','cum_returns'])